In [10]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
from matplotlib.patches import Patch

def get_direction_code(img_sitk):
    """ Get the direction of a SimpleITK image.
    Args:
        img_sitk (SimpleITK.Image): Input image.
    Returns:
        str: Direction code of the image.
    """
    direction_code = sitk.DICOMOrientImageFilter().GetOrientationFromDirectionCosines(img_sitk.GetDirection())

    return direction_code

def reorient_sitk(img_sitk, new_direction):
    """ Reorient a SimpleITK image to a new direction.
    Args:
        img_sitk (SimpleITK.Image): Input image to be reoriented.
        new_direction (str): New direction code (e.g., 'LPS', 'RAS').
    Returns:
        SimpleITK.Image: Reoriented image.
    """

    img_sitk_reoriented = sitk.DICOMOrient(img_sitk, new_direction)


In [ ]:
import os
from tqdm import tqdm
import numpy as np
import SimpleITK as sitk

def dice_score(y_true, y_pred, smooth=1e-6):
    """
    Compute the Dice score between two binary masks.

    Args:
        y_true (numpy.ndarray): Ground truth binary mask.
        y_pred (numpy.ndarray): Predicted binary mask.
        smooth (float): Smoothing factor to avoid division by zero.

    Returns:
        float: Dice score.
    """
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

label_gt_path = "/scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/labelsTs"
label_pred_path = "/scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/imagesTs/man_preds"

label_gt_file = [x for x in os.listdir(label_gt_path) if x.endswith(".nii.gz")]
label_pred_file = [x for x in os.listdir(label_pred_path) if x.endswith(".nii.gz")]

all_subjects = [x.split('_')[0] for x in label_pred_file]

#Make sure the files are sorted in the same order
label_gt_file = sorted(label_gt_file, key=lambda x: x.split('_')[0])
label_pred_file = sorted(label_pred_file, key=lambda x: x.split('_')[1])

# if len(label_gt_file) != len(label_pred_file):
#     raise ValueError("Number of ground truth and prediction files do not match")

output_folder = "/scratch/awias/data/Pancreas/test"

dice_list = []
n = len(label_gt_file)

for subject in all_subjects:
    try:
        mask_gt_sitk = sitk.ReadImage(os.path.join(label_gt_path, f"{subject}.nii.gz"))
        pred_sitk = sitk.ReadImage(os.path.join(label_pred_path, f"{subject}_0000_pred.nii.gz"))
        pred_sitk.CopyInformation(mask_gt_sitk)  # Ensure the prediction has the same metadata as the ground truth
        
        # Save mask_gt_sitk and pred_sitk with new orientation in same folder
        sitk.WriteImage(mask_gt_sitk, os.path.join(output_folder, f"{subject}_reoriented.nii.gz"))
        sitk.WriteImage(pred_sitk, os.path.join(output_folder, f"{subject}_0000_pred_reoriented.nii.gz"))

        print("Saved")

        mask_gt = sitk.GetArrayFromImage(mask_gt_sitk)
        pred = sitk.GetArrayFromImage(pred_sitk)
        
        print(get_direction_code(mask_gt_sitk))
        print(get_direction_code(pred_sitk))

        # print(os.path.join(label_gt_path,label_gt_file[i]))
        # print(os.path.join(label_pred_path,label_pred_file[i]))
        # plot_central_slice_img_mask(mask_gt, pred, spacing=None)
        

        dice = dice_score(mask_gt, pred)
        dice_list.append(dice)
        
        print(f"Dice score for subject {subject} is: {dice}")
    except:
        print(f"Error processing subject {subject}. Skipping...")

print(f"Mean dice score: {sum(dice_list) / len(dice_list)}")
print(f"Median dice score: {sorted(dice_list)[len(dice_list) // 2]}")
print(f"Max dice score: {max(dice_list)}")
print(f"Min dice score: {min(dice_list)}")


LAI
LAS
Dice score for subject 38 is: 0.1871655113265129
LAI
LAS
Dice score for subject 13 is: 0.4692161940795918
LAI
LAS
Dice score for subject 39 is: 0.6036707511242038
LAI
LAS
Dice score for subject 72 is: 0.4233411100878321
LAI
LAS
Dice score for subject 73 is: 0.49599690750693226
LAI
LAS
Dice score for subject 1 is: 0.053517184521711086
LAI
LAS
Dice score for subject 75 is: 0.05186112087621442
LAI
LAS
Dice score for subject 51 is: 0.31401815284298235
LAI
LAS
Dice score for subject 19 is: 0.4675092890271181
LAI
LAS


NiftiImageIO (0x4501ae0): /scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/labelsTs/27.nii.gz has unexpected scales in sform

NiftiImageIO (0x4501ae0): /scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/labelsTs/27.nii.gz has unexpected scales in sform



Dice score for subject 27 is: 0.20780566130834238
LAI
LAS
Dice score for subject 30 is: 0.4443571283738746
LAI
LAS
Dice score for subject 55 is: 0.4762363070617162
LAI
LAS
Dice score for subject 36 is: 0.17663482991292026
LAI
LAS
Error processing subject 40. Skipping...
LAI
LAS
Dice score for subject 20 is: 0.4280424517621527
LAI
LAS
Dice score for subject 42 is: 0.07563502465494715
Mean dice score: 0.3250005082978035
Median dice score: 0.4233411100878321
Max dice score: 0.6036707511242038
Min dice score: 0.05186112087621442
